In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import re
from textwrap import wrap
import matplotlib.ticker
import os
import seaborn as sns


In [ ]:
def append_vars(ds, cols):
    v=ds[cols]
    X=ds.drop(columns=cols)
    return pd.concat([X.assign(cfu=ds[col], name=ds['year'].apply(lambda y: f"{col} {y}")) for col in cols]).dropna()

In [ ]:
import matplotlib.ticker as mtick
def make_dfs(df,fname):
    colors = ['#00008b', '#1e90ff', '#40e0d0', 'g', 'y']
    fig, ax = plt.subplots(figsize=(12,8))
    sns.set_palette(colors)
    ax=sns.swarmplot(x='week', y='cfu', hue='name', data=df,  alpha=0.8, size=12, linewidth=1, edgecolor='w')
    #ax=sns.scatterplot(x='week', y='cfu', hue='name', data=c,  alpha=0.8, size= 14, linewidth=1, edgecolor='w')
    plt.axvline(x=1.42, color='b', linestyle='-.', alpha=0.8)
    plt.axvline(x=3.7, color='b', linestyle='-.', alpha=0.8)
    ax.text(2.56, 1960,  "Jazz 2016", ha="center", va="center", size=14, color='b')
    ax.annotate("", xy=(1.42, 1900), xycoords='data',
               xytext=(3.7, 1900), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='b', alpha=0.8))
    plt.axvline(x=2.4, color='r', linestyle='-.', alpha=0.8)
    plt.axvline(x=4.58, color='r', linestyle='-.', alpha=0.8)
    ax.text(3.4, 2240,  "Jazz 2017", ha="center", va="center", size=14, color='r')
    ax.annotate("", xy=(2.4, 2200), xycoords='data',
               xytext=(4.58, 2200), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='r', alpha=0.8))
    ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
    ax.set_xlabel("Sample weeks 2016, 2017 and 2020", size=16, labelpad=20)
    ax.set_ylabel("Colony forming units per 100mL", size=16, labelpad=20)
    plt.suptitle("Montreux large events and the occurrence of bacteria in lake water" , fontsize=16, family='sans')
    plt.title("Colonies grouped by year and week number", fontsize=16, family='sans', y=1.03)
    ax.legend(loc='upper right', fontsize=14)
    plt.savefig(f"{fname}.svg")
    plt.savefig(f"{fname}.png")

In [ ]:
A=pd.read_pickle("data/allyears-avg_plate.pkl")

In [ ]:
make_dfs(append_vars(A,['Bioindicator', 'Bioindicator UV+', 'Coliform']),
        'plots/YEAR_OVER_YEAR_CFUINCREASE_JAZZ_2016_2017_2020')